In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, min, max, rand, randn
from faker import Faker

In [17]:
# fake = Faker()
tags = fake.coordinate(center=None, radius=0.001)
print(tags)


def f(x):
    return fake.coordinate(center=None, radius=0.001)

list(map(lambda x: f(x), list(range(10))))

-139.507999


[Decimal('172.816876'),
 Decimal('-13.879081'),
 Decimal('-69.892454'),
 Decimal('4.431388'),
 Decimal('-128.536729'),
 Decimal('146.388684'),
 Decimal('172.841787'),
 Decimal('151.936922'),
 Decimal('-127.637413'),
 Decimal('151.866261')]

#### Create Spark Session (SQL Context)

In [5]:
spark = SparkSession \
        .builder \
        .appName('demo') \
        .getOrCreate()

#### Create Spark Dataframe

In [6]:
# alt ways to create a spark dataframe
#alt_way_a = spark.createDataFrame(["10","11","13"], "string").toDF("age")
#alt_way_b = spark.createDataFrame([("10", ), ("11", ), ("13",  )], ["age"])
#from pyspark.sql.types import StringType
#spark.createDataFrame(["10","11","13"], StringType()).toDF("age")

In [7]:
df = spark \
        .range(0, 10) \
        .withColumn('uniform', rand(seed=13)) \
        .withColumn('normal', randn(seed=31)) \
        .withColumn('normal', randn(seed=31))

#### Add Row

In [9]:
new_row = spark.createDataFrame([(10, 1, 1)], ['id', 'uniform', 'normal'])
df = df.union(new_row)
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- uniform: double (nullable = true)
 |-- normal: double (nullable = true)



#### Add Col

In [11]:
df = df.withColumn('apd', rand(seed=11))
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- uniform: double (nullable = true)
 |-- normal: double (nullable = true)
 |-- apd: double (nullable = false)



In [21]:
df.select([mean('uniform'), min('uniform'), max('uniform')]).show()

+------------------+-------------------+------------+
|      avg(uniform)|       min(uniform)|max(uniform)|
+------------------+-------------------+------------+
|0.4803029184230118|0.06415762022697513|         1.0|
+------------------+-------------------+------------+



In [14]:
# sample covariance of two columns
df.stat.cov('uniform', 'normal')

-0.06948283612954509

#### Create Spark RDD

In [65]:
# events
e_cols = ["EVENTID","EVENTTIMESTAMP","EVENTTYPE","PROVIDERID","PATIENTID","TASKID","ENCOUNTERID","WORKSTATIONID","PROCESS_ID","EVENTACTION"]
e_rows = [
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","10003000","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","99001647","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","99001647","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","20000073","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","","prd-32835774","QUERY")
]

# workstations
w_cols = ["WORKSTATIONID","WORKSTATIONAREA","EXTERNAL_NAME","DEPARTMENT_ID","NETWORK","COMPANY"]
w_rows = [
    ("10003000","BDW ORTHO/NEURO","Brandywine Hospital Ortho/Neuro","107001009","Brandywine","Brandywine Hospital"),
    ("20000073","FAMILY FLOURTOWN THMG","Springfield Medical Associates","102409001","Chestnut Hill","CHN - Chestnut Hill Clinic"),
    ("00195B2FF60701080B160C03020E1809","EMH EMERGENCY","EMH Emergency Department","10101100","",""),
    ("40001111","PHX 2AR","Rehabilitation at Phoenixville Hospital","107601028","Phoenixville","Phoenixville Hospital"),
    ("50001297","PTN MAIN OR","Pottstown Main OR","107801016","Pottstown","Pottstown Hospital"),
    ("50001303","PTN EMERGENCY","Pottstown Hospital Emergency Department","107801001","Pottstown","Pottstown Hospital"),
    ("50001310","INTL MED BOYER THMG","Berks Medical Associates","103201001","Pottstown","PTS - Pottstown Clinic Co")
]

e_df = spark.createDataFrame(e_rows, e_cols)
w_df = spark.createDataFrame(w_rows, w_cols)

lte_df = w_df.select(w_df['WORKSTATIONID'], w_df['NETWORK'])

In [66]:
e_df.printSchema()
e_df.show()

root
 |-- EVENTID: string (nullable = true)
 |-- EVENTTIMESTAMP: string (nullable = true)
 |-- EVENTTYPE: string (nullable = true)
 |-- PROVIDERID: string (nullable = true)
 |-- PATIENTID: string (nullable = true)
 |-- TASKID: string (nullable = true)
 |-- ENCOUNTERID: string (nullable = true)
 |-- WORKSTATIONID: string (nullable = true)
 |-- PROCESS_ID: string (nullable = true)
 |-- EVENTACTION: string (nullable = true)

+-----------------+-------------------+---------+----------+---------+------+-----------+-------------+------------+-----------+
|          EVENTID|     EVENTTIMESTAMP|EVENTTYPE|PROVIDERID|PATIENTID|TASKID|ENCOUNTERID|WORKSTATIONID|  PROCESS_ID|EVENTACTION|
+-----------------+-------------------+---------+----------+---------+------+-----------+-------------+------------+-----------+
|5636059882.643001|2019-08-07 00:11:22|     View|     14173|  Z854894| 17008|30041094735|     10003000|prd-32835774|      QUERY|
|5636059882.643001|2019-08-07 00:11:22|     View|     1417

In [ ]:
foo = w_df.filter()select(w_df['WORKSTATIONID'], w_df['NETWORK'])

In [67]:
newRow = spark.createDataFrame([("4", "ASD")], ['WORKSTATIONID', 'NETWORK'])
appended = lte_df.union(newRow)
appended.show()

# .filter(df.age > 3)

+--------------------+-------------+
|       WORKSTATIONID|      NETWORK|
+--------------------+-------------+
|            10003000|   Brandywine|
|            20000073|Chestnut Hill|
|00195B2FF60701080...|             |
|            40001111| Phoenixville|
|            50001297|    Pottstown|
|            50001303|    Pottstown|
|            50001310|    Pottstown|
|                   4|          ASD|
+--------------------+-------------+



In [ ]:
w_df.printSchema()
w_df.show()

In [68]:
df = e_df.join(lte_df, "WORKSTATIONID", "right_outer")

In [75]:
df.filter(df['EVENTID'] != '').show()

+-------------+-----------------+-------------------+---------+----------+---------+------+-----------+------------+-----------+-------------+
|WORKSTATIONID|          EVENTID|     EVENTTIMESTAMP|EVENTTYPE|PROVIDERID|PATIENTID|TASKID|ENCOUNTERID|  PROCESS_ID|EVENTACTION|      NETWORK|
+-------------+-----------------+-------------------+---------+----------+---------+------+-----------+------------+-----------+-------------+
|     20000073|5636059882.643001|2019-08-07 00:11:22|     View|     14173|  Z854894| 17008|30041094735|prd-32835774|      QUERY|Chestnut Hill|
|     10003000|5636059882.643001|2019-08-07 00:11:22|     View|     14173|  Z854894| 17008|30041094735|prd-32835774|      QUERY|   Brandywine|
+-------------+-----------------+-------------------+---------+----------+---------+------+-----------+------------+-----------+-------------+



In [ ]:
from pyspark.sql.functions import round
a = df.groupBy('NETWORK').count()

def pctVal(x):
    #return round(a['count']/df.count(),2)
    b = round(x/df.count(),2)
    return b
    
a.withColumn('pct', round(a['count']/df.count(), 2)).show()

In [ ]:
def func(v):
    print(a[v])
    return v

a.select('NETWORK', func('count')).show()